In [ ]:
import gensim.downloader as api

- 'word2vec-google-news-300'
- 'fasttext-wiki-news-subwords-300'
- 'glove-wiki-gigaword-50': Global Vectors for Word Representation, antrenat pe date de Wikipedia din 2014, cuvintele sunt reprezentate in 50 de dimensiuni

In [ ]:
# # Reprezentarile vectoriale pentru modelul pre-trained word2vec
# word2vec = api.load('word2vec-google-news-300')

In [ ]:
# Obtinem reprezentarile vectoriale din modelul pre-antrenat glove
glove = api.load('glove-wiki-gigaword-50')

In [ ]:
glove['order']

In [ ]:
# Similarity ne returneaza cosine similarity:
# - o valoare apropiata de 1 indica vectori similari (sensuri asemanatoare)
# - o valoare de 0 indica vectori independenti
# - o valoare de 1 indica vectori similari, dar opusi
glove.similarity('good', 'great')

In [ ]:
glove.similarity('seaside', 'abstract')

In [ ]:
glove.similarity('terrible', 'awful')

In [ ]:
# Top 10 cuvinte care au cel mai mare scor de similaritate cu "great"
glove.most_similar('great', topn=10)

In [ ]:
glove.most_similar(positive=['computer', 'human'], negative=['interface'])

In [ ]:
glove.doesnt_match(['fire', 'water', 'air', 'car'])